# Information Retrieval

This notebook demonstrates how to put together a simple retrieval engine in python. To start we'll focus on *boolean retrieval*, which works over bags of words. We won't bother about any optimisations, and just use python dicts, sets etc.

Our dataset is from NLTK, using the Reuters document collection. You may need to call `nltk.download()` to install the Reuters and Stopwords corpora. 

In [1]:
import nltk
corpus = nltk.corpus.reuters
from math import log
from collections import defaultdict, Counter

Take a look at the data, which are text documents like the one below.

In [2]:
print corpus.raw(corpus.fileids()[0])

ASIAN EXPORTERS FEAR DAMAGE FROM U.S.-JAPAN RIFT
  Mounting trade friction between the
  U.S. And Japan has raised fears among many of Asia's exporting
  nations that the row could inflict far-reaching economic
  damage, businessmen and officials said.
      They told Reuter correspondents in Asian capitals a U.S.
  Move against Japan might boost protectionist sentiment in the
  U.S. And lead to curbs on American imports of their products.
      But some exporters said that while the conflict would hurt
  them in the long-run, in the short-term Tokyo's loss might be
  their gain.
      The U.S. Has said it will impose 300 mln dlrs of tariffs on
  imports of Japanese electronics goods on April 17, in
  retaliation for Japan's alleged failure to stick to a pact not
  to sell semiconductors on world markets at below cost.
      Unofficial Japanese estimates put the impact of the tariffs
  at 10 billion dlrs and spokesmen for major electronics firms
  said they would virtually halt exports

We'll need to tokenise the documents, remove stop-words and stem the words to form our bag-of-words representation. Here we'll use the PorterStemmer (there are others in NLTK). 

In [3]:
stopwords = set(nltk.corpus.stopwords.words('english')) # wrap in a set() (see below)
stemmer = nltk.stem.PorterStemmer() 

def extract_terms(doc):
    terms = set()
    for token in nltk.word_tokenize(doc):
        if token not in stopwords: # 'in' and 'not in' operations are much faster over sets that lists
            terms.add(stemmer.stem(token.lower()))
    return terms

Let's test the method using the first document.

In [4]:
doc = corpus.raw(corpus.fileids()[0])
print list(sorted(extract_terms(doc)))

[u'&', u"''", u"'s", u'(', u')', u',', u'.', u'10', u'15.6', u'17', u'1985', u'30', u'300', u'4.9', u'53', u'7.1', u'95', u';', u'>', u'``', u'a', u'abl', u'account', u'action', u'advantag', u'alleg', u'allow', u'also', u'american', u'among', u'analyst', u'and', u'april', u'asia', u'asian', u'ask', u'associ', u'australia', u'australian', u'avow', u'await', u'awar', u'barrier', u'beef', u'below-cost', u'beyond', u'biggest', u'billion', u'block', u'boost', u'broker', u'budget', u'busi', u'businessmen', u'but', u'button', u'call', u'canberra', u'capel', u'capit', u'centr', u'chairman', u'chief', u'co', u'coal', u'commerci', u'complet', u'concern', u'conflict', u'continu', u'correspond', u'cost', u'could', u'countri', u'curb', u'cut', u'damag', u'day', u'defus', u'democrat', u'deputi', u'despit', u'deterior', u'diplomat', u'director-gener', u'disadvantag', u'disput', u'dlr', u'domest', u'due', u'econom', u'economi', u'effort', u'electr', u'electron', u'emerg', u'end', u'eros', u'estim', u'

We probably want to remove numbers and punctuation, which aren't being caught by the stop list. We may want to be a bit more agressive with tokenising hyphenated words (although take care, as some might be important.) Have a go yourself and see if you can improve the preprocessing to correct for these issues.

Now we can apply the term extraction method to all documents in our corpus. (This may take a minute or two.) 

In [5]:
docs = {}
for docid in corpus.fileids():
    terms = extract_terms(corpus.raw(docid))
    docs[docid] = terms

And build an inverted index, which transposes the above data structure such that terms are the key and a set of document identifiers are the values.

In [6]:
inverted_index = defaultdict(list)
for docid, terms in docs.items():
    for term in terms:
        inverted_index[term].append(docid)
        
# ensure posting lists are in sorted order
for term, docids in inverted_index.items():
    docids.sort()

Let's try a query term, say *'trade'*. We can retreive all documents containing this term (being careful to process the query in the same way as the document).

In [7]:
inverted_index[stemmer.stem('trade'.lower())]

['test/14826',
 'test/14829',
 'test/14832',
 'test/14839',
 'test/14840',
 'test/14858',
 'test/14860',
 'test/14862',
 'test/14863',
 'test/14873',
 'test/14881',
 'test/14885',
 'test/14886',
 'test/14891',
 'test/14900',
 'test/14904',
 'test/14912',
 'test/14913',
 'test/14931',
 'test/14951',
 'test/14957',
 'test/14982',
 'test/14987',
 'test/15043',
 'test/15096',
 'test/15103',
 'test/15112',
 'test/15121',
 'test/15129',
 'test/15154',
 'test/15156',
 'test/15171',
 'test/15198',
 'test/15206',
 'test/15223',
 'test/15246',
 'test/15262',
 'test/15287',
 'test/15290',
 'test/15306',
 'test/15310',
 'test/15313',
 'test/15352',
 'test/15367',
 'test/15372',
 'test/15375',
 'test/15386',
 'test/15389',
 'test/15396',
 'test/15413',
 'test/15421',
 'test/15424',
 'test/15428',
 'test/15430',
 'test/15442',
 'test/15446',
 'test/15447',
 'test/15449',
 'test/15450',
 'test/15452',
 'test/15454',
 'test/15460',
 'test/15468',
 'test/15472',
 'test/15484',
 'test/15501',
 'test/155

How about a multiple term query? Consider *U.S. free trade* as our query.

In [8]:
postings1 = inverted_index[stemmer.stem('U.S.'.lower())]
postings2 = inverted_index[stemmer.stem('free'.lower())]
postings3 = inverted_index[stemmer.stem('trade'.lower())]
print len(postings1), len(postings2), len(postings3)

1987 172 1506


We now have to intersect the posting lists (sets here) to implement the conjuctive query. As the postings are in sorted order, we can do this efficiently, two at a time:

In [9]:
def intersect_postings(postings1, postings2):
    i = j = 0
    results = []
    while i < len(postings1) and j < len(postings2):
        if postings1[i] < postings2[j]:
            i += 1
        elif postings1[i] > postings2[j]:
            j += 1
        else:
            results.append(postings1[i])
            i += 1
            j += 1
    return results

Can you see why the postings lists need to be sorted? Think about what the above algorithm might look like otherwise, and its time complexity.

Now we can test it on our query. Note that as we have 3 posting lists, we could do (1 and 2) then merge the result with 3, or another order. The order will affect the runtime, so we will process them in order of length, from smallest to largest. 

In [10]:
# more efficient order
result23 = intersect_postings(postings2, postings3)
result123 = intersect_postings(postings1, result23)
print len(result23), len(result123)

# less efficient order
result12 = intersect_postings(postings1, postings2)
result123 = intersect_postings(result12, postings3)
print len(result12), len(result123)

81 47
85 47


Note that when there are many results, as here, all these documents are judged to be equally relevant, because they all contain all of the query terms.

In [11]:
print corpus.raw(result123[0])

JAPAN IN LAST-DITCH EFFORT TO AVERT TARIFFS
  Senior Japanese officials tomorrow
  open talks with American trade negotiators in a last-ditch
  effort to avert new high U.S. tariffs to be imposed on a wide
  variety of Japanese electronic exports.
      Makoto Kuroda, vice minister of Japan's Ministry of
  International Trade and Industry (MITI), is to hold two days of
  meetings with the Deputy U.S. Trade Representative, Michael
  Smith, and the Under Secretary of Commerce, Bruce Smart.
      The new tariffs, to go into effect on April 17, are in
  retaliation for Japan's failure to adhere to an agreement to
  end dumping semiconductors in world markets at below cost and
  to open its home market to U.S. semiconductor shipments.
      They are to be imposed on goods which use semiconductors,
  including television and audio equipment and computers.
      Both U.S. and Japanese officials have said there was little
  likelihood the talks would do anything to avert the 100 pct
  duties o

You might want to think about how to process queries that include more terms, and disjunctions (OR) or negations.

# Vector space model of IR

Now we will consider a _vector space model_ approach to IR, where we define a scoring function capable of relating a query to a document, then use this to retrieve the top *k* scoring documents for a given query. The scoring function we use is the cosine similarity measure over a _TF*IDF_ vector representation of the document and the query. 

This requires us to process the data to compute:
1. term frequencies within each document (a *bag* of words, rather than a *set* as for boolean retrieval)
2. document frequencies for each term, counting how many documents they occur in
3. normlised _tf*idf_ vectors for each document 
4. an inverted index to allow for efficient lookup by term

The first step is to collect term frequencies for each document, which is only a slight change from the code above for creating the binary index.

In [12]:
def extract_term_freqs(doc):
    tfs = Counter()
    for token in nltk.word_tokenize(doc):
        if token not in stopwords: # 'in' and 'not in' operations are much faster over sets that lists
            tfs[stemmer.stem(token.lower())] += 1
    return tfs

We'll also need to compute the document frequencies, for the *idf* factors in the scoring function.

In [13]:
def compute_doc_freqs(doc_term_freqs):
    dfs = Counter()
    for tfs in doc_term_freqs.values():
        for term in tfs.keys():
            dfs[term] += 1
    return dfs

Using the above two functions, process the corpus into term frequencies and document frequencies.

In [14]:
doc_term_freqs = {}
for docid in corpus.fileids():
    term_freqs = extract_term_freqs(corpus.raw(docid))
    doc_term_freqs[docid] = term_freqs
M = len(doc_term_freqs)

In [15]:
doc_freqs = compute_doc_freqs(doc_term_freqs)

As a sanity check, let's verify that our query terms have the correct *df* values, matching the number of results in the posting lists from above.

In [16]:
for term in "U.S. free trade".split():
    stem = stemmer.stem(term.lower())
    print "term", term, "df", doc_freqs[stem], 'idf', log(M / float(doc_freqs[stem]))

term U.S. df 1987 idf 1.69180844171
term free df 172 idf 4.13869520745
term trade df 1506 idf 1.9689772759


Now to create the inverted index, which is a bit more involved as we need to include the scoring function to compute the *tf* and *idf* values, then normalise each document vector to be unit length. For this we process each document in the corpus, compute a vector with one entry per term, normalise for length and store in the inverted index.

Note that this is the inefficient inverted index, as described in lecture 16. Many optimisations are possible, and are indeed necessary for this to scale to more realistic sized datasets. The approach below is adequate for small collections. 

In [17]:
vsm_inverted_index = defaultdict(list)
for docid, term_freqs in doc_term_freqs.items():
    N = sum(term_freqs.values())
    length = 0
    
    # find tf*idf values and accumulate sum of squares 
    tfidf_values = []
    for term, count in term_freqs.items():
        tfidf = float(count) / N * log(M / float(doc_freqs[term]))
        tfidf_values.append((term, tfidf))
        length += tfidf ** 2

    # normalise documents by length and insert into index
    length = length ** 0.5
    for term, tfidf in tfidf_values:
        # note the inversion of the indexing, to be term -> (doc_id, score)
        vsm_inverted_index[term].append([docid, tfidf / length])
        
# ensure posting lists are in sorted order (less important here cf above)
for term, docids in vsm_inverted_index.items():
    docids.sort()

For querying this index, we use the algorithm from lecture 16, slide 12. This just sums up the weights for each document using the posting lists for all query terms, then returns the ranked list of results. (Again, there are more efficient algorithms for doing ranked retrieval in the VSM, such as [WAND](http://cis.poly.edu/westlab/papers/cntdstrb/p426-broder.pdf).)

In [18]:
def query_vsm(query, index, k=10):
    accumulator = Counter()
    for term in query:
        postings = index[term]
        for docid, weight in postings:
            accumulator[docid] += weight
    return accumulator.most_common(k)

results = query_vsm([stemmer.stem(term.lower()) for term in "U.S. free trade".split()], vsm_inverted_index)
results

[('training/9315', 0.5891114990355987),
 ('training/4509', 0.5606677802020277),
 ('training/12558', 0.5571337933449025),
 ('training/14739', 0.5441759307505476),
 ('training/2352', 0.5362306820102838),
 ('training/4552', 0.5165780430886306),
 ('training/5376', 0.5155325673064437),
 ('training/12563', 0.5142876470281071),
 ('test/18992', 0.511001222477988),
 ('training/4533', 0.5021341664457657)]

It's worth taking a look at the top scoring document(s), e.g.

In [19]:
print corpus.raw(results[0][0])

KEY U.S. HOUSE PANEL FINISHES MAJOR TRADE BILL
  The House Ways and Means Committee
  completed action on legislation to toughen U.S. trade laws,
  chairman Dan Rostenkowski said.
      The committee's consideration of one of the most
  controversial provisions, a plan to force major trade surplus
  countries to cut their trade imbalance with the United States,
  was deferred until the full House considers the trade bill, its
  sponsor Rep. Richard Gephardt said.
      Gephardt, a Missouri Democrat, told Reuters he was not
  certain the exact form his trade surplus reduction proposal
  would take. Last year the House approved his plan to force a 10
  pct surplus cutback each year for four years, by countries such
  as Japan.
      The Ways and Means Committess' trade bill forces President
  Reagan to retaliate against unfair trade practices that violate
  international trade agreements but it allows him to wave
  retaliatory tariffs or quotas if the action would hurt the U.S.
  economy

Note that this document has a very high TF for *U.S.* and *trade* but does not include *free*, despite it being the highest *idf* entry in the query.